# Modeling an Electron in a Field of Secured Point Charges

While it can be easy to visualize in your head or on paper the movements of an electron when there is only a couple of charges and when all of the charges are positive or negative, it's a lot harder to figure out where an electron will end up by hand. This model is to help students understand how electrons move under the influence of multiple charges of varying amounts of charge.

In [10]:
import math #basic python math library, used for square rooting and getting pi
#https://docs.python.org/3/library/math.html#
import numpy as np #common library, source of numpy arrays which are easier to manipulate as vectors than your average python list
#https://numpy.org/doc/stable/index.html

from scipy.constants import e, m_e, epsilon_0 
#physics constants I need, the charge of a proton or electron (e), the mass of an electron (m_e), and epsilon naught
#https://docs.scipy.org/doc/scipy/reference/constants.html

import matplotlib.pyplot as plt #for plotting data
#https://matplotlib.org/3.5.3/api/_as_gen/matplotlib.pyplot.html
import matplotlib.animation as animation #to create animation
#https://matplotlib.org/stable/api/animation_api.html
from IPython.display import HTML #making it possible to display my animation
#https://ipython.readthedocs.io/en/stable/api/generated/IPython.display.html

The first thing that we need to do is import libraries beyond basic python to give us the capabilities that we need to make an animation and do the necessary math to determine where the electron goes. 

The first library we import is math, which is a built-in python library that contains functions like sqrt() (square root) and constants like pi. 

The next library is numpy, which has ndarrays, which are a type of list known as an array where you can perform mathematical operations. With a normal list, adding them results in a new list that has elements from both lists, but with ndarrays you end up with a ndarray of the same length where you add the first elements of both arrays, then the second, and so on.

After that we important a couple of constants from the scipy library, which has fundamental algorithms for scientific computing and a variety of scientific constants. 

Next, we have matplotlib.pyplot and matplotlib.animation. pyplot is needed for the plotting (graph making) of my project, while animation allows for me to turn these plots into models through animating the motion of the elctron.

Lastly, there's HTML from IPython.display, which will allow me to display my animation in the frontend, making it visible to users.

In [11]:
class Charge: #class to create Charge objects, which will make it easier to work with charges

    def __init__(self, x, y, charge): #initiation function, magic method
        self.position = np.array([float(x), float(y)]) ##creating the position, using the given x and y coordinates, as an array
        #https://numpy.org/doc/stable/reference/generated/numpy.array.html
        self.charge = charge #getting the charge of the particle

    def get_position(self): #function to get the position of a charge
        return self.position #return the position

    def get_charge(self): #function to get the charge of a charge
        return self.charge #return the charge

Explain what on object is in coding.

This charge object is used to have a way to represent every point charge an their elements. Charges have two elements, their position and their charge, so it's not a very complicated object, but it makes it possible to hold all the elements of a charge together.

In [12]:
class Electron: #class to create Electron objects, which will allow me to make functions for electrons

    def __init__(self, x, y): #initiation function, magic method
        self.position = np.array([float(x), float(y)]) #creating the position, using the given x and y coordinates, as an array
        self.charge = -e #all electrons have a charge of -e
        self.velocity = np.array([0.0, 0.0]) #start with 0 velocity
        self.acceleration = np.array([0.0, 0.0]) #start with 0 acceleration

    def get_position(self): #function to get the position of an electrno
        return self.position #return the position

    def get_acceleration(self): #return the acceleration
        return self.acceleration #return acceleration

    def get_velocity(self):
        return self.velocity

    def get_force(self, charges): #function to get the sum of all the forces on an electron
        force = np.array([0.0, 0.0]) #start with 0
        for charge in charges: #then go through each charge
            r = math.sqrt((charge.get_position() - self.get_position())[0] ** 2 + (charge.get_position() - self.get_position())[1] ** 2)
            #get the distance from electron to charge
            force += (self.get_position() - charge.get_position()) * charge.get_charge() * self.charge / 4 / math.pi / epsilon_0 / (r ** 3)
            #use Coloumb's Law to find the force a single charge places on the electron and add it to the current force
        self.acceleration = force / m_e / 100 #Newton's second law, f=ma, so a=f/m

    def update_velocity(self): #changing velocity
        self.velocity += self.acceleration #small increments, velocity changes by acceleration

    def update_position(self): #changing position
        self.position += self.velocity #small increments, position changes by velocity

The electron is a much more complex object. Just like a charge, it also has a position and a charge, but it also has an acceleration and a velocity because it is not fixed in place. Additionally, it has more than just an initiation method and methods to get the values held by an electron. If has a method to calculate the force on it, taking both itself and all the charges present. Force starts at a numpy array with 0s, then it goes through each charge and adds the force that charge represents using Coloumb's Law. Afterwards, acceleration is gotten through dividing by the mass of an electron. There's also methods to update velocity and position, done by adding acceleration and velocity resepectively. As velocity is the area under the curve of acceleration, when you just update slowly, you can get velocity by adding up all the accerations, so when force is calculated, the next step is to call to update velocity. The same thing happens with position.

In [13]:
def electron_update(charges, electron): #function to update the state of the electron

    electron.get_force(charges) #getting the current force
    electron.update_velocity() #updating velocity
    electron.update_position() #updating position

This is the function used when going to the next point in the animation. First, the force is found and the acceleration is set. We update velocity, adding acceleration to the current velocity. Then we update position by adding the current velocity to the current position. Nothing is returned because all of these functions just effect the electron and nothing is calculated that needs to be returned in this function.

In [14]:
 i = 1 #start with one frame

def gen(charges, electron): #making a generator function!
    global i #letting us access i, prevents it from resetting
    sep = True #boolean to determine if you should continue
    while sep:
        #print(i)
        for charge in charges: #look at each charge
            if abs(electron.get_position()[0] - charge.get_position()[0]) < 2 and abs(electron.get_position()[1] - charge.get_position()[1]) < 2:
            #if the electron is practically on the charge
                sep = False #don't continue
                print("on charge") #let us know we are on a charge
                print(charge.get_position(), electron.get_position()) #print both locations
                print(abs(electron.get_position()[0] - charge.get_position()[0]), abs(electron.get_position()[1] - charge.get_position()[1])) #print distance
                break #exit this loop
        if abs(electron.get_position()[0]) > 100 or abs(electron.get_position()[1]) > 100: #if it's outside of the boundaries of the simulation
            print("left boundaries")
            sep = False #don't continue
        if np.array_equal(electron.get_velocity(), [0.0,0.0]) and np.array_equal(electron.get_acceleration(), [0.0, 0.0]): #if the velocity + acceleration is 0
            print("no velocity/acceleration") #let us know that the velocity/acceleration is 0
            print(electron.get_velocity(), electron.get_acceleration()) #prove it
            sep = False #don't continue
        if sep == True: #if we still should continue
            i += 1 #up the frame number
            #print(i)
            yield i #iteration version of return

This is a generation function. Matplotlib animations use them when you want the animation to stop at a specified action instead of at a specific number of frames. As long as the yield goes, the animation will continue, if the generator doesn't yield anything, the animation will stop updating. This yield number counts up in order to keep track of the number of frames. The animation stops if the electron is touching any of teh charges, it the electron leaves the bounds of the animation, or if both the velocity and the acceleration of the electron equal 0.

In [15]:
def create_animation(charges, electron): #creating the animation

    fig, ax = plt.subplots() #needed elements of the plot
    #https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.subplots.html
    
    scatter = ax.scatter([c.get_position()[0] for c in charges if c.get_charge() > 0], [c.get_position()[1] for c in charges if c.get_charge() > 0], c="r", label="+ charges")
    #placing all of the positive charges on the plot, coloring them red
    #https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.scatter.html#matplotlib.axes.Axes.scatter
    scatter = ax.scatter([c.get_position()[0] for c in charges if c.get_charge() < 0], [c.get_position()[1] for c in charges if c.get_charge() < 0 ], c="b", label="- charges")
    #placing all of the negative charges on the plot, coloring them blue
    scatter = ax.scatter(electron.get_position()[0], electron.get_position()[1], c="g", label="electron")
    #place the electron on the plot, coloring it green

    ax.set_xlim([-100, 100]) #making the x-axis -100 to 100
    #https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.set_xlim.html
    ax.set_ylim([-100, 100]) #making the y-axis -100 to 100
    #https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.set_ylim.html
    ax.legend() #show the legend
    #https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.legend.html
    
    def update(frame): #update function!

        electron_update(charges, electron) #call the update function to set the values
        scatter.set_offsets(electron.get_position()) #update the position of the electron
        #https://matplotlib.org/stable/api/collections_api.html#matplotlib.collections.AsteriskPolygonCollection.set_offsets
        return scatter #return the new scatter plot

    electron.get_force(charges) #not have starting acceleration 0
    anim = animation.FuncAnimation(fig=fig, func=update, frames=gen(charges, electron), interval=10, save_count=1000) #create the animation
    #https://matplotlib.org/stable/api/_as_gen/matplotlib.animation.FuncAnimation.html
    plt.close() #close the plot

    return HTML(anim.to_jshtml()) #return the HTML to make the animation
    #https://matplotlib.org/stable/api/_as_gen/matplotlib.animation.Animation.html#matplotlib.animation.Animation.to_jshtml

Here is the function that actually creates the animation. The first thing it does is create a graph, setting it as a figure with axes. The next thing it does is place the charges, starting with positive charges, coloring them red. Then it places the negative charges, coloring them blue. Finally, the electron is red. Next, it makes the graph (-100, 100) by (-100, 100) and shows the legend. Then, inside the animation creation function, there's the update function for the animation, where it updates the electron, moves the position of the electron on the scatter plot through .set_offsets, then return the new plot. Before creating the animation, we get the force so the animation doesn't immediately stop because there's no acceleration and velocity. Then, the animation is created, using the figure we set up, our update function and generator function. We end by closing the plot so we are able to release the plot in a way that lets us actually see it on the frontend.

In [16]:
print("This program will create a 2d simulation with point charges fixed in place and an unsecured electron.")
print("The dimensions are (-100, 100) by (-100, 100).")
print()
print("First, create your Charges.")
q = int(input("Number of Charges (int): "))
charges = []
for i in range(q):
    x = float(input("Enter x coordinate (float): "))
    y = float(input("Enter y coordinate (float): "))
    c = float(input("Enter charge (float): "))
    charges.append(Charge(x, y, c * e))
print("Second, locate your Electron.")
x = float(input("Enter x coordinate (float): "))
y = float(input("Enter y coordinate (float): "))
electron = Electron(x, y)
create_animation(charges, electron)

This program will create a 2d simulation with point charges fixed in place and an unsecured electron.
The dimensions are (-100, 100) by (-100, 100).

First, create your Charges.


Number of Charges (int):  4
Enter x coordinate (float):  30
Enter y coordinate (float):  40
Enter charge (float):  3
Enter x coordinate (float):  -30
Enter y coordinate (float):  40
Enter charge (float):  3
Enter x coordinate (float):  0
Enter y coordinate (float):  30
Enter charge (float):  -3
Enter x coordinate (float):  -30
Enter y coordinate (float):  -40
Enter charge (float):  3


Second, locate your Electron.


Enter x coordinate (float):  0
Enter y coordinate (float):  0


on charge
[-30. -40.] [-31.20305311 -41.26332035]
1.2030531076891684 1.2633203514722595


Here, there's a set of print statements that explain what the program does, asks for input of what charges there are and where they are, and the position of the electron. Then it makes the animation!